# exp036

In [45]:
import os
import sys
import gc
import itertools
import pickle
import pathlib
import datetime
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv
load_dotenv()
sys.path.append(os.getenv('UTILS_PATH'))

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import optuna

import line_notify

In [46]:
import builtins
import types

def imports():
    for name, val in globals().items():
        # module imports
        if isinstance(val, types.ModuleType):
            yield name, val

            # functions / callables
        if hasattr(val, '__call__'):
            yield name, val


def noglobal(f):
    '''
    ref: https://gist.github.com/raven38/4e4c3c7a179283c441f575d6e375510c
    '''
    return types.FunctionType(f.__code__,
                              dict(imports()),
                              f.__name__,
                              f.__defaults__,
                              f.__closure__
                              )

In [47]:
SEED = 42
N_ITER = 1
RUN_INF = False # 推論処理を行うか
BATCH_SIZE = int(5e5)

In [48]:
Ns = {}
Ns['cf_a'] = 12
Ns['ctf_a'] = 12
Ns['atfd_a'] = 12
Ns['atfp_a'] = 12
Ns['pa_a'] = 12

Ns['cf_w'] = 12
Ns['ctf_w'] = 12
Ns['atfd_w'] = 12
Ns['atfp_w'] = 12
Ns['pa_w'] = 12

Ns['cf_m'] = 12
Ns['ctf_m'] = 12
Ns['atfd_m'] = 12
Ns['atfp_m'] = 12
Ns['pa_m'] = 12

Ns['cf_y'] = 12
Ns['ctf_y'] = 12
Ns['atfd_y'] = 12
Ns['atfp_y'] = 12
Ns['pa_y'] = 12

ディレクトリ設定

In [49]:
INPUT_DIR = os.getenv('INPUT_DIR')
OUTPUT_DIR = os.getenv('OUTPUT_DIR')
#exp_name = os.path.dirname(__file__).split('/')[-1]
exp_name = 'exp036'
os.makedirs(OUTPUT_DIR + exp_name, exist_ok=True)

データ読み込み

In [50]:
articles = pd.read_csv(INPUT_DIR + 'articles.csv', dtype='object')
customers = pd.read_csv(INPUT_DIR + 'customers.csv')
transactions = pd.read_csv(INPUT_DIR + 'transactions_train.csv', dtype={'article_id':'str'}, parse_dates=['t_dat'])
sample = pd.read_csv(INPUT_DIR + 'sample_submission.csv')

# 前処理

In [51]:
ALL_CUSTOMER = customers['customer_id'].unique().tolist()
ALL_ARTICLE = articles['article_id'].unique().tolist()

customer_ids = dict(list(enumerate(ALL_CUSTOMER)))
article_ids = dict(list(enumerate(ALL_ARTICLE)))

customer_map = {u: uidx for uidx, u in customer_ids.items()}
article_map = {i: iidx for iidx, i in article_ids.items()}

articles['article_id'] = articles['article_id'].map(article_map)
customers['customer_id'] = customers['customer_id'].map(customer_map)
transactions['article_id'] = transactions['article_id'].map(article_map)
transactions['customer_id'] = transactions['customer_id'].map(customer_map)
sample['customer_id'] = sample['customer_id'].map(customer_map)

In [52]:
# 名寄せ
customers['fashion_news_frequency'] = customers['fashion_news_frequency'].str.replace('None','NONE')

In [53]:
customers['age10'] = str((customers['age'] // 10) * 10)
customers.loc[customers['age'].isnull(), 'age10'] = np.nan

In [54]:
# label_encoding
le_cols = ['product_type_name', 'product_group_name', 'graphical_appearance_name',
            'colour_group_name', 'perceived_colour_value_name', 'perceived_colour_master_name', 'department_name',
            'index_name', 'index_group_name', 'section_name', 'garment_group_name']
for c in le_cols:
    le = LabelEncoder()
    articles[c] = le.fit_transform(articles[c].fillna(''))


le_cols = ['club_member_status', 'fashion_news_frequency', 'postal_code', 'age10']
for c in le_cols:
    le = LabelEncoder()
    customers[c] = le.fit_transform(customers[c].fillna(''))

In [55]:
customers['customer_type'] = customers['FN'].fillna(0).astype(int).astype(str) + \
                             customers['Active'].fillna(0).astype(int).astype(str) + \
                             customers['club_member_status'].fillna(0).astype(int).astype(str) + \
                             customers['fashion_news_frequency'].fillna(0).astype(int).astype(str) + \
                             customers['age10'].fillna(0).astype(int).astype(str)

le = LabelEncoder()
customers['customer_type'] = le.fit_transform(customers['customer_type'])

In [56]:
# transactionに紐づけ
transactions = transactions.merge(customers, on='customer_id', how='left')
transactions = transactions.merge(articles, on='article_id', how='left')

# データセット作成（レコメンド→対象データセット作成→特徴量エンジニアリング）

In [57]:
@noglobal
def get_customer_frequent(history, n=12, timedelta=None):
    """顧客ごと商品の購入数をカウントし上位の商品を抽出

    Args:
        history (dataframe): 集計対象の実績データ
        n (int): レコメンド対象とする数
        timedelta (dateutil.relativedelta): 指定された場合、実績データの終端からtimedelta分のデータを取得する

    Returns:
        dataframe: 抽出結果
    """
    if timedelta is not None:
        st_date = history['t_dat'].max() - timedelta
        history = history[history['t_dat']>=st_date].copy()
        
    customer_agg = history.groupby(['customer_id', 'article_id'])['t_dat'].count().reset_index()
    customer_agg = customer_agg.rename(columns={'t_dat':'cnt'})
    customer_agg = customer_agg.sort_values(['customer_id', 'cnt'], ascending=False)
    result = customer_agg.groupby('customer_id').head(n)
    return result[['customer_id', 'article_id']]

@noglobal
def get_popular_article(history, n=12, timedelta=None):
    """全体の購入数をカウントし上位の商品を抽出

    Args:
        history (dataframe): 集計対象の実績データ
        n (int): レコメンド対象とする数
        timedelta (dateutil.relativedelta): 指定された場合、実績データの終端からtimedelta分のデータを取得する

    Returns:
        list: 抽出結果
    """
    # 全体の購入数量
    if timedelta is not None:
        st_date = history['t_dat'].max() - timedelta
        history = history[history['t_dat']>=st_date].copy()

    total_agg = history.groupby('article_id')['t_dat'].count().reset_index()
    total_agg = total_agg.rename(columns={'t_dat':'cnt'})
    total_agg = total_agg.sort_values(['cnt'], ascending=False)
    total_agg = total_agg.head(n)
    result = list(total_agg['article_id'].values)
    return result

@noglobal
def get_customer_type_frequent(history, n=12, timedelta=None):
    if timedelta is not None:
        st_date = history['t_dat'].max() - timedelta
        history = history[history['t_dat']>=st_date].copy()

    result = history[['customer_id', 'customer_type']].drop_duplicates().copy()
    agg = history.groupby(['customer_type', 'article_id'])['t_dat'].count().reset_index()
    agg = agg.rename(columns={'t_dat':'cnt'})
    agg = agg.sort_values(['customer_type', 'cnt'], ascending=False)
    agg = agg.groupby('customer_type').head(n)
    result = result.merge(agg[['customer_type', 'article_id']], on='customer_type', how='left')
    return result[['customer_id', 'article_id']]

@noglobal
def get_article_type_frequent(history, col, n=12, timedelta=None):
    if timedelta is not None:
        st_date = history['t_dat'].max() - timedelta
        history = history[history['t_dat']>=st_date].copy()

    result = history.groupby(['customer_id', col])['t_dat'].count().reset_index()
    result = result.rename(columns={'t_dat':'cnt'})
    result = result.sort_values(['customer_id', 'cnt'], ascending=False)
    result = result.groupby(['customer_id']).head(1)[['customer_id', col]]

    agg = history.groupby([col, 'article_id'])['t_dat'].count().reset_index()
    agg = agg.rename(columns={'t_dat':'cnt'})
    agg = agg.sort_values([col, 'cnt'], ascending=False)
    agg = agg.groupby(col).head(n)
    result = result.merge(agg[[col, 'article_id']], on=col, how='left')
    return result[['customer_id', 'article_id']]

@noglobal
def get_reccomend(target_customer_id, history, Ns):
    n = 12
    result = pd.DataFrame()
    

    td = None
    result = result.append(get_customer_frequent(history, Ns['cf_a'], td))
    result = result.append(get_customer_type_frequent(history, Ns['ctf_a'], td))
    result = result.append(get_article_type_frequent(history, 'department_name', Ns['atfd_a'], td))
    result = result.append(get_article_type_frequent(history, 'perceived_colour_master_name', Ns['atfp_a'], td))
    popular_article = get_popular_article(history, Ns['pa_a'], td)
    # customerとpopular articleの全組み合わせでdataframe作成
    popular_article = pd.DataFrame(itertools.product(target_customer_id, popular_article), columns=['customer_id', 'article_id'])
    result = result.append(popular_article)
    result = result.drop_duplicates()

    td = relativedelta(weeks=1)
    result = result.append(get_customer_frequent(history, Ns['cf_w'], td))
    result = result.append(get_customer_type_frequent(history, Ns['ctf_w'], td))
    result = result.append(get_article_type_frequent(history, 'department_name', Ns['atfd_w'], td))
    result = result.append(get_article_type_frequent(history, 'perceived_colour_master_name', Ns['atfp_w'], td))
    popular_article = get_popular_article(history, Ns['pa_w'], td)
    # customerとpopular articleの全組み合わせでdataframe作成
    popular_article = pd.DataFrame(itertools.product(target_customer_id, popular_article), columns=['customer_id', 'article_id'])
    result = result.append(popular_article)
    result = result.drop_duplicates()

    td = relativedelta(months=1)
    result = result.append(get_customer_frequent(history, Ns['cf_m'], td))
    result = result.append(get_customer_type_frequent(history, Ns['ctf_m'], td))
    result = result.append(get_article_type_frequent(history, 'department_name', Ns['atfd_m'], td))
    result = result.append(get_article_type_frequent(history, 'perceived_colour_master_name', Ns['atfp_m'], td))
    popular_article = get_popular_article(history, Ns['pa_m'], td)
    # customerとpopular articleの全組み合わせでdataframe作成
    popular_article = pd.DataFrame(itertools.product(target_customer_id, popular_article), columns=['customer_id', 'article_id'])
    result = result.append(popular_article)
    result = result.drop_duplicates()

    td = relativedelta(years=1)
    result = result.append(get_customer_frequent(history, Ns['cf_y'], td))
    result = result.append(get_customer_type_frequent(history, Ns['ctf_y'], td))
    result = result.append(get_article_type_frequent(history, 'department_name', Ns['atfd_y'], td))
    result = result.append(get_article_type_frequent(history, 'perceived_colour_master_name', Ns['atfp_y'], td))
    popular_article = get_popular_article(history, Ns['pa_y'], td)
    # customerとpopular articleの全組み合わせでdataframe作成
    popular_article = pd.DataFrame(itertools.product(target_customer_id, popular_article), columns=['customer_id', 'article_id'])
    result = result.append(popular_article)
    result = result.drop_duplicates()

    result = result[result['customer_id'].isin(target_customer_id)].copy()

    return result

In [58]:
@noglobal
def add_labels(recom_result, history):
    """レコメンドしたデータが学習期間で購入されたかどうかのフラグを付与する

    Args:
        recom_result (_type_): レコメンド結果
        train_tran (_type_): 学習期間のトランザクションデータ

    Returns:
        _type_: 学習期間での購入フラグを付与したレコメンド結果
    """
    history = history[['customer_id', 'article_id']].drop_duplicates()
    history['buy'] = 1
    recom_result = recom_result.merge(history, on=['customer_id', 'article_id'], how='left')
    recom_result['buy'] = recom_result['buy'].fillna(0)
    return recom_result


In [59]:
@noglobal
def make_article_features(articles):
    cols = ['product_type_name', 'product_group_name', 'graphical_appearance_name',
            'colour_group_name', 'perceived_colour_value_name', 'perceived_colour_master_name', 'department_name',
            'index_name', 'index_group_name', 'section_name', 'garment_group_name']
    return articles[['article_id']+cols]

@noglobal
def make_article_tran_features(history):
    df = history.groupby('article_id').agg({'t_dat':['count', 'max', 'min'],
                                            'price':['max', 'min', 'mean'], 
                                            'age':['max', 'min', 'mean', 'std']}).reset_index()
    df.columns = ['article_id','article_total_cnt', 'article_total_latest_buy', 'article_total_1st_buy', 'article_price_max', 'article_price_min', 'article_price_mean', 'article_age_max', 'article_age_min', 'article_age_mean', 'article_age_std']
    df['article_total_1st_buy'] = (history['t_dat'].max() - df['article_total_1st_buy']).dt.days
    df['article_total_latest_buy'] = (history['t_dat'].max() - df['article_total_latest_buy']).dt.days

    for i in range(2):
        istr = str(i+1)

        history_weekago = history.loc[(history['t_dat'] > history['t_dat'].max() - relativedelta(days=(i+1)*7)) & 
                                    (history['t_dat'] <= history['t_dat'].max() - relativedelta(days=i*7))]

        history_weekago_df = history_weekago.groupby('article_id').agg({'t_dat':['count', 'max', 'min'],
                                                'price':['max', 'min', 'mean'], 
                                                'age':['max', 'min', 'mean', 'std', 'median']}).reset_index()
        history_weekago_df.columns = ['article_id',f'article_total_cnt_{istr}weekago', f'article_total_latest_buy_{istr}weekago', f'article_total_1st_buy_{istr}weekago', f'article_price_max_{istr}weekago', f'article_price_min_{istr}weekago', 
                                      f'article_price_mean_{istr}weekago', f'article_age_max_{istr}weekago', f'article_age_min_{istr}weekago', f'article_age_mean_{istr}weekago', f'article_age_std_{istr}weekago', f'article_age_median_{istr}weekago']
        history_weekago_df[f'article_total_1st_buy_{istr}weekago'] = (history_weekago['t_dat'].max() - history_weekago_df[f'article_total_1st_buy_{istr}weekago']).dt.days
        history_weekago_df[f'article_total_latest_buy_{istr}weekago'] = (history_weekago['t_dat'].max() - history_weekago_df[f'article_total_latest_buy_{istr}weekago']).dt.days

        df = pd.merge(df,history_weekago_df,how='left',on='article_id')

    return df


@noglobal
def make_customer_features(customers):
    return customers

@noglobal
def make_customer_tran_features(history):
    group = ['Ladieswear', 'Divided', 'Menswear', 'Sport', 'Baby/Children']
    for g in group:
        history[g] = 0
        history.loc[history['index_group_name']==g, g] = 1


    df = history.groupby('customer_id').agg({'t_dat':['count', 'max', 'min'],
                                            'price':['max', 'min', 'mean'],
                                            'Ladieswear':'sum',
                                            'Divided':'sum',
                                            'Menswear':'sum',
                                            'Sport':'sum',
                                            'Baby/Children':'sum'}).reset_index()
    df.columns = ['customer_id','customer_total_cnt', 'customer_total_latest_buy', 'customer_total_1st_buy', 
                  'customer_price_max', 'customer_price_min', 'customer_price_mean',
                  'Ladieswear', 'Divided', 'Menswear', 'Sport', 'Baby/Children']
    df['customer_total_1st_buy'] = (history['t_dat'].max() - df['customer_total_1st_buy']).dt.days
    df['customer_total_latest_buy'] = (history['t_dat'].max() - df['customer_total_latest_buy']).dt.days

    for g in group:
        df[g] = df[g] / df['customer_total_cnt']

#####################iida_exp24#########################
    for i in range(2):
        istr = str(i+1)
        history_weekago = history.loc[(history['t_dat'] > history['t_dat'].max() - relativedelta(days=(i+1)*7)) & 
                                    (history['t_dat'] <= history['t_dat'].max() - relativedelta(days=i*7))]
        history_weekago_df =  history_weekago.groupby('customer_id').agg({'t_dat':['count', 'max', 'min'],
                                                'price':['max', 'min', 'mean'],
                                                'Ladieswear':'sum',
                                                'Divided':'sum',
                                                'Menswear':'sum',
                                                'Sport':'sum',
                                                'Baby/Children':'sum'}).reset_index() 
        history_weekago_df.columns = ['customer_id',f'customer_total_cnt_{istr}weekago', f'customer_total_latest_buy_{istr}weekago', f'customer_total_1st_buy_{istr}weekago', 
                                    f'customer_price_max_{istr}weekago', f'customer_price_min_{istr}weekago', f'customer_price_mean_{istr}weekago',
                                    f'Ladieswear_{istr}weekago', f'Divided_{istr}weekago', f'Menswear_{istr}weekago', f'Sport_{istr}weekago', f'Baby/Children_{istr}weekago']

        history_weekago_df[f'customer_total_1st_buy_{istr}weekago'] = (history_weekago['t_dat'].max() - history_weekago_df[f'customer_total_1st_buy_{istr}weekago']).dt.days
        history_weekago_df[f'customer_total_latest_buy_{istr}weekago'] = (history_weekago['t_dat'].max() - history_weekago_df[f'customer_total_latest_buy_{istr}weekago']).dt.days

        for g in group:
            history_weekago_df[g+f'_{istr}weekago'] = history_weekago_df[g+f'_{istr}weekago'] / history_weekago_df[f'customer_total_cnt_{istr}weekago']

        df = pd.merge(df,history_weekago_df,how='left',on='customer_id')

    return df

@noglobal
def make_customer_article_features(target, history):
    df = target.merge(history, on=['customer_id', 'article_id'], how='inner')
    df = df.groupby(['customer_id', 'article_id']).agg({'t_dat':['count', 'min', 'max']}).reset_index()
    df.columns = ['customer_id', 'article_id', 'count', '1st_buy_date_diff', 'latest_buy_date_diff']
    df['1st_buy_date_diff'] = (history['t_dat'].max() - df['1st_buy_date_diff']).dt.days
    df['latest_buy_date_diff'] = (history['t_dat'].max() - df['latest_buy_date_diff']).dt.days
    return df

@noglobal
def add_same_article_type_rate(target, history, col):
    add_data = history[['customer_id', col]].copy()
    add_data['total'] = add_data.groupby('customer_id').transform('count')
    add_data = add_data.groupby(['customer_id', col])['total'].agg(['max', 'count']).reset_index()
    add_data[f'{col}_customer_buy_rate'] = add_data['count'] / add_data['max']
    target = target.merge(add_data[['customer_id', col, f'{col}_customer_buy_rate']], on=['customer_id', col], how='left')
    return target

    

@noglobal
def add_features(df, history, articles, customers):
    df = df.merge(make_article_features(articles), on=['article_id'], how='left')
    df = df.merge(make_article_tran_features(history), on=['article_id'], how='left')
    df = df.merge(make_customer_features(customers), on=['customer_id'], how='left')
    df = df.merge(make_customer_tran_features(history), on=['customer_id'], how='left')
    df = df.merge(make_customer_article_features(df[['customer_id', 'article_id']], history), on=['article_id', 'customer_id'], how='left')

    cols = ['product_type_name', 'product_group_name', 'graphical_appearance_name',
            'colour_group_name', 'perceived_colour_value_name', 'perceived_colour_master_name', 'department_name',
            'index_name', 'index_group_name', 'section_name', 'garment_group_name']

    for c in cols:
        df = add_same_article_type_rate(df, history, c)

    return df
    

# レコメンド商品を購入するかどうかの2値分類モデル

In [60]:
def apk(y_true, y_pred, K=12):
    assert(len(y_true) == len(y_pred))
    apks = []
    for idx in range(len(y_true)):
        y_i_true = y_true[idx]
        y_i_pred = y_pred[idx]

        # 予測値の数と重複の確認
        assert(len(y_i_pred) <= K)
        assert(len(np.unique(y_i_pred)) == len(y_i_pred))

        sum_precision = 0.0
        num_hits = 0.0

        for i, p in enumerate(y_i_pred):
            if p in y_i_true:
                num_hits += 1
                precision = num_hits / (i+1)
                sum_precision += precision
        apk = sum_precision / min(len(y_i_true), K)
        apks.append(apk)
    return apks

In [61]:
@noglobal
def run_train(transactions, articles, customers, Ns):

    # 1週ずつローリングして学習データを生成し検証
    train_start = datetime.datetime(2020,9,9)
    valid_start = datetime.datetime(2020,9,16)
    valid_end = datetime.datetime(2020,9,22)

    # 学習データの作成
    history_tran = transactions[transactions['t_dat'] < train_start].copy()
    target_tran = transactions[(transactions['t_dat'] >= train_start) & (transactions['t_dat'] < valid_start)].copy()
    target_id = target_tran['customer_id'].unique().tolist()
    recom = get_reccomend(target_id, history_tran, Ns)
    ml_train = add_labels(recom, target_tran)
    ml_train = add_features(ml_train, history_tran, articles, customers)

    # 評価データの作成
    history_tran = transactions[transactions['t_dat'] < valid_start].copy()
    target_tran = transactions[(transactions['t_dat'] >= valid_start) & (transactions['t_dat'] <= valid_end)].copy()
    target_id = target_tran['customer_id'].unique().tolist()
    recom = get_reccomend(target_id, history_tran, Ns)
    ml_valid = add_labels(recom, target_tran)
    ml_valid = add_features(ml_valid, history_tran, articles, customers)
    
    target = 'buy'
    not_use_cols = ['customer_id', 'article_id', target]
    features = [c for c in ml_train.columns if c not in not_use_cols]

    params = {
    "objective": "binary",
    "boosting" : "gbdt",
    "learning_rate": 0.01,
    "metric": "binary_logloss",
    "seed": 42
    }  

    # 学習
    tr_x, tr_y = ml_train[features], ml_train[target]
    vl_x, vl_y = ml_valid[features], ml_valid[target]
    tr_data = lgb.Dataset(tr_x, label=tr_y)
    vl_data = lgb.Dataset(vl_x, label=vl_y)
    model = lgb.train(params, tr_data, valid_sets=[tr_data, vl_data],
                    num_boost_round=20000, early_stopping_rounds=100,verbose_eval=1000)

    # cv
    vl_pred = model.predict(vl_x, num_iteration=model.best_iteration)
    # 正解データ作成
    valid = transactions[(transactions['t_dat'] >= valid_start) & (transactions['t_dat'] <= valid_end)].copy()
    valid = valid[['customer_id', 'article_id']].drop_duplicates()
    valid = valid.groupby('customer_id')['article_id'].apply(list).reset_index()
    valid = valid.sort_values('customer_id').reset_index(drop=True)
    # 2値分類の出力を元に12個選定
    valid_pred = ml_valid[['customer_id', 'article_id']].copy()
    valid_pred['prob'] = vl_pred
    valid_pred = valid_pred.sort_values(['customer_id', 'prob'], ascending=False)
    valid_pred = valid_pred.groupby('customer_id').head(12)
    valid_pred = valid_pred.groupby('customer_id')['article_id'].apply(list).reset_index()
    valid_pred = valid_pred.sort_values('customer_id').reset_index(drop=True)
    assert(valid['customer_id'].tolist() == valid_pred['customer_id'].tolist())
    # MAP@12
    score = np.mean(apk(valid['article_id'].tolist(), valid_pred['article_id'].tolist()))
    
    return score

In [62]:
def objective(trial):
    cf_a = trial.suggest_int('cf_a', 0, 24)
    ctf_a = trial.suggest_int('ctf_a', 0, 24)
    atfd_a = trial.suggest_int('atfd_a', 0, 24)
    atfp_a = trial.suggest_int('atfp_a', 0, 24)
    pa_a = trial.suggest_int('pa_a', 0, 24)

    cf_w = trial.suggest_int('cf_w', 0, 24)
    ctf_w = trial.suggest_int('ctf_w', 0, 24)
    atfd_w = trial.suggest_int('atfd_w', 0, 24)
    atfp_w = trial.suggest_int('atfp_w', 0, 24)
    pa_w = trial.suggest_int('pa_w', 0, 24)

    cf_m = trial.suggest_int('cf_m', 0, 24)
    ctf_m = trial.suggest_int('ctf_m', 0, 24)
    atfd_m = trial.suggest_int('atfd_m', 0, 24)
    atfp_m = trial.suggest_int('atfp_m', 0, 24)
    pa_m = trial.suggest_int('pa_m', 0, 24)

    cf_y = trial.suggest_int('cf_y', 0, 24)
    ctf_y = trial.suggest_int('ctf_y', 0, 24)
    atfd_y = trial.suggest_int('atfd_y', 0, 24)
    atfp_y = trial.suggest_int('atfp_y', 0, 24)
    pa_y = trial.suggest_int('pa_y', 0, 24)

    Ns['cf_a'] = cf_a
    Ns['ctf_a'] = ctf_a
    Ns['atfd_a'] = atfd_a
    Ns['atfp_a'] = atfp_a
    Ns['pa_a'] = pa_a

    Ns['cf_w'] = cf_w
    Ns['ctf_w'] = ctf_w
    Ns['atfd_w'] = atfd_w
    Ns['atfp_w'] = atfp_w
    Ns['pa_w'] = pa_w

    Ns['cf_m'] = cf_m
    Ns['ctf_m'] = ctf_m
    Ns['atfd_m'] = atfd_m
    Ns['atfp_m'] = atfp_m
    Ns['pa_m'] = pa_m

    Ns['cf_y'] = cf_y
    Ns['ctf_y'] = ctf_y
    Ns['atfd_y'] = atfd_y
    Ns['atfp_y'] = atfp_y
    Ns['pa_y'] = pa_y

    total_n = ctf_a + pa_a + \
              ctf_w + pa_w + \
              ctf_m + pa_m + \
              ctf_y + pa_y

    if total_n > 12:
        score = run_train(transactions, articles, customers, Ns)
    else:
        score = 0.0
    message = f'{Ns}\n{score}'
    line_notify.send(message)

    return score

In [63]:
study = optuna.create_study(direction='maximize')
study.optimize(objective)

[I 2022-03-07 00:48:36,209] A new study created in memory with name: no-name-5fd0d51e-50f9-4096-99eb-f48303976df1
/home/kaggler/.local/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/kaggler/.local/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 16604, number of negative: 5158704
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.488675 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 12454
[LightGBM] [Info] Number of data points in the train set: 5175308, number of used features: 98
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.003208 -> initscore=-5.738797
[LightGBM] [Info] Start training from score -5.738797
Training until validation scores don't improve for 100 rounds
[1000]	training's binary_logloss: 0.0168993	valid_1's binary_logloss: 0.0198174
Early stopping, best iteration is:
[1084]	training's binary_logloss: 0.0168016	valid_1's binary_logloss: 0.0198157


[I 2022-03-07 01:06:23,526] Trial 0 finished with value: 0.029945630398482054 and parameters: {'cf_a': 13, 'ctf_a': 4, 'atfd_a': 2, 'atfp_a': 6, 'pa_a': 2, 'cf_w': 2, 'ctf_w': 14, 'atfd_w': 3, 'atfp_w': 11, 'pa_w': 20, 'cf_m': 4, 'ctf_m': 7, 'atfd_m': 0, 'atfp_m': 12, 'pa_m': 15, 'cf_y': 24, 'ctf_y': 23, 'atfd_y': 5, 'atfp_y': 3, 'pa_y': 0}. Best is trial 0 with value: 0.029945630398482054.
/home/kaggler/.local/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/kaggler/.local/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument 

[LightGBM] [Info] Number of positive: 20565, number of negative: 7755649
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.799833 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 12436
[LightGBM] [Info] Number of data points in the train set: 7776214, number of used features: 98
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.002645 -> initscore=-5.932586
[LightGBM] [Info] Start training from score -5.932586
Training until validation scores don't improve for 100 rounds
[1000]	training's binary_logloss: 0.0142973	valid_1's binary_logloss: 0.0162176
Early stopping, best iteration is:
[1099]	training's binary_logloss: 0.0142039	valid_1's binary_logloss: 0.0162155


[I 2022-03-07 01:30:44,729] Trial 1 finished with value: 0.031803011799666825 and parameters: {'cf_a': 21, 'ctf_a': 16, 'atfd_a': 18, 'atfp_a': 17, 'pa_a': 23, 'cf_w': 12, 'ctf_w': 17, 'atfd_w': 20, 'atfp_w': 19, 'pa_w': 23, 'cf_m': 20, 'ctf_m': 17, 'atfd_m': 1, 'atfp_m': 1, 'pa_m': 0, 'cf_y': 23, 'ctf_y': 17, 'atfd_y': 18, 'atfp_y': 12, 'pa_y': 0}. Best is trial 1 with value: 0.031803011799666825.
/home/kaggler/.local/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/kaggler/.local/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' a

...

[LightGBM] [Info] Number of positive: 20488, number of negative: 8343553
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.820125 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 12404
[LightGBM] [Info] Number of data points in the train set: 8364041, number of used features: 98
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.002450 -> initscore=-6.009405
[LightGBM] [Info] Start training from score -6.009405
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[497]	training's binary_logloss: 0.0138169	valid_1's binary_logloss: 0.0151585


[I 2022-03-08 06:39:12,728] Trial 102 finished with value: 0.03169046486318756 and parameters: {'cf_a': 22, 'ctf_a': 24, 'atfd_a': 17, 'atfp_a': 9, 'pa_a': 15, 'cf_w': 4, 'ctf_w': 12, 'atfd_w': 0, 'atfp_w': 16, 'pa_w': 19, 'cf_m': 16, 'ctf_m': 4, 'atfd_m': 5, 'atfp_m': 24, 'pa_m': 15, 'cf_y': 23, 'ctf_y': 11, 'atfd_y': 23, 'atfp_y': 22, 'pa_y': 11}. Best is trial 53 with value: 0.0321293428107754.
/home/kaggler/.local/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/kaggler/.local/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' ar

[LightGBM] [Info] Number of positive: 19188, number of negative: 7596686
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.701233 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 12405
[LightGBM] [Info] Number of data points in the train set: 7615874, number of used features: 98
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.002519 -> initscore=-5.981182
[LightGBM] [Info] Start training from score -5.981182
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[475]	training's binary_logloss: 0.0140294	valid_1's binary_logloss: 0.0154752


[I 2022-03-08 06:53:30,306] Trial 103 finished with value: 0.03165038522262257 and parameters: {'cf_a': 21, 'ctf_a': 13, 'atfd_a': 14, 'atfp_a': 12, 'pa_a': 11, 'cf_w': 2, 'ctf_w': 8, 'atfd_w': 2, 'atfp_w': 13, 'pa_w': 17, 'cf_m': 17, 'ctf_m': 6, 'atfd_m': 4, 'atfp_m': 19, 'pa_m': 14, 'cf_y': 24, 'ctf_y': 9, 'atfd_y': 24, 'atfp_y': 22, 'pa_y': 10}. Best is trial 53 with value: 0.0321293428107754.


KeyboardInterrupt: 

In [64]:
study.best_params

{'cf_a': 21,
 'ctf_a': 20,
 'atfd_a': 15,
 'atfp_a': 6,
 'pa_a': 11,
 'cf_w': 4,
 'ctf_w': 18,
 'atfd_w': 18,
 'atfp_w': 14,
 'pa_w': 18,
 'cf_m': 20,
 'ctf_m': 5,
 'atfd_m': 5,
 'atfp_m': 15,
 'pa_m': 18,
 'cf_y': 23,
 'ctf_y': 9,
 'atfd_y': 21,
 'atfp_y': 22,
 'pa_y': 12}